In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from lazypredict.Supervised import LazyClassifier

In [2]:
# load dataset

train = pd.read_csv("./rs-data-science-wise-202324/train.csv")
test = pd.read_csv("./rs-data-science-wise-202324/test.csv")

In [3]:
train.head()

,UniqueID,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Soil_Type_40,Cover_Type
0,0,3291.00,358.00,18.00,30.00,10.00,2161.00,189.00,204.00,150.00,...,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1
1,1,2563.00,334.00,18.00,60.00,5.00,1084.00,177.00,214.00,174.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2
2,2,3340.00,187.00,24.00,90.00,21.00,1041.00,215.00,NaN,154.00,...,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,7
3,3,2922.00,103.00,4.00,67.00,3.00,5057.00,227.00,234.00,141.00,...,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,1
4,4,2728.00,15.00,15.00,446.00,86.00,NaN,205.00,207.00,136.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2


In [4]:
train.columns

Index(['UniqueID', 'Elevation', 'Aspect', 'Slope',
       'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
       'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon',
       'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points',
       'Wilderness_Area_1', 'Wilderness_Area_2', 'Wilderness_Area_3',
       'Wilderness_Area_4', 'Soil_Type_1', 'Soil_Type_2', 'Soil_Type_3',
       'Soil_Type_4', 'Soil_Type_5', 'Soil_Type_6', 'Soil_Type_7',
       'Soil_Type_8', 'Soil_Type_9', 'Soil_Type_10', 'Soil_Type_11',
       'Soil_Type_12', 'Soil_Type_13', 'Soil_Type_14', 'Soil_Type_15',
       'Soil_Type_16', 'Soil_Type_17', 'Soil_Type_18', 'Soil_Type_19',
       'Soil_Type_20', 'Soil_Type_21', 'Soil_Type_22', 'Soil_Type_23',
       'Soil_Type_24', 'Soil_Type_25', 'Soil_Type_26', 'Soil_Type_27',
       'Soil_Type_28', 'Soil_Type_29', 'Soil_Type_30', 'Soil_Type_31',
       'Soil_Type_32', 'Soil_Type_33', 'Soil_Type_34', 'Soil_Type_35',
       'Soil_Type_36', 'Soil

In [5]:
# extract binary features
binary_features = [col for col in train.columns if train[col].nunique() == 2]

# extract categorical features (i.e., integer features with <= 10 unique values)
categorical_features = [
    col for col in train.columns if train[col].dtype == "int64" and train[col].nunique() <= 10
]

# extract numerical features
numerical_features = [
    col for col in train.columns if train[col].dtype in ["int64", "float64"] and col not in binary_features
]


In [6]:
numerical_features

['UniqueID',
 'Elevation',
 'Aspect',
 'Slope',
 'Horizontal_Distance_To_Hydrology',
 'Vertical_Distance_To_Hydrology',
 'Horizontal_Distance_To_Roadways',
 'Hillshade_9am',
 'Hillshade_Noon',
 'Hillshade_3pm',
 'Horizontal_Distance_To_Fire_Points',
 'Soil_Type_15',
 'Cover_Type']

In [7]:
# remove id, soil type and cover type from numerical features
numerical_features.remove("UniqueID")
numerical_features.remove("Cover_Type")
numerical_features.remove("Soil_Type_15")

In [8]:
categorical_features

['Cover_Type']

In [9]:
binary_features

['Wilderness_Area_1',
 'Wilderness_Area_2',
 'Wilderness_Area_3',
 'Wilderness_Area_4',
 'Soil_Type_1',
 'Soil_Type_2',
 'Soil_Type_3',
 'Soil_Type_4',
 'Soil_Type_5',
 'Soil_Type_6',
 'Soil_Type_7',
 'Soil_Type_8',
 'Soil_Type_9',
 'Soil_Type_10',
 'Soil_Type_11',
 'Soil_Type_12',
 'Soil_Type_13',
 'Soil_Type_14',
 'Soil_Type_16',
 'Soil_Type_17',
 'Soil_Type_18',
 'Soil_Type_19',
 'Soil_Type_20',
 'Soil_Type_21',
 'Soil_Type_22',
 'Soil_Type_23',
 'Soil_Type_24',
 'Soil_Type_25',
 'Soil_Type_26',
 'Soil_Type_27',
 'Soil_Type_28',
 'Soil_Type_29',
 'Soil_Type_30',
 'Soil_Type_31',
 'Soil_Type_32',
 'Soil_Type_33',
 'Soil_Type_34',
 'Soil_Type_35',
 'Soil_Type_36',
 'Soil_Type_37',
 'Soil_Type_38',
 'Soil_Type_39',
 'Soil_Type_40']

In [10]:
train['Soil_Type_15'].value_counts()

Soil_Type_15
0.00    9634
Name: count, dtype: int64

In [11]:
# add soil type 15 to binary features
binary_features.append('Soil_Type_15')

In [12]:
soil_type_features = [col for col in train.columns if col.startswith('Soil_Type')]
wilderness_area_features = [col for col in train.columns if col.startswith('Wilderness_Area')]


In [13]:
# remove cover type from categorical features
categorical_features.remove('Cover_Type')

In [14]:
# drop nan rows
train = train.dropna()

In [15]:
# take the binary features of the soil type and make them one categorical feature
train['Soil_Type'] = train[soil_type_features].idxmax(axis=1)

# take the binary features of the wilderness area and make them one categorical feature
train['Wilderness_Area'] = train[wilderness_area_features].idxmax(axis=1)

# drop the binary features of the soil type
train = train.drop(soil_type_features, axis=1)

# drop the binary features of the wilderness area
train = train.drop(wilderness_area_features, axis=1)

In [16]:
train.head()

,UniqueID,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Cover_Type,Soil_Type,Wilderness_Area
0,0,3291.00,358.00,18.00,30.00,10.00,2161.00,189.00,204.00,150.00,1405.00,1,Soil_Type_38,Wilderness_Area_3
1,1,2563.00,334.00,18.00,60.00,5.00,1084.00,177.00,214.00,174.00,577.00,2,Soil_Type_29,Wilderness_Area_1
7,7,3077.00,64.00,13.00,60.00,12.00,5079.00,232.00,212.00,110.00,757.00,1,Soil_Type_30,Wilderness_Area_1
10,10,2407.00,300.00,22.00,182.00,63.00,1024.00,155.00,225.00,209.00,1518.00,6,Soil_Type_10,Wilderness_Area_4
11,11,2801.00,237.00,18.00,30.00,5.00,2507.00,186.00,253.00,201.00,470.00,2,Soil_Type_13,Wilderness_Area_3


In [17]:
X = train.drop(['Cover_Type', 'UniqueID'], axis=1)
y = train['Cover_Type']

In [18]:
# split data into train and validation sets
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
# convert categorical features to string and binary features to bool
#for col in categorical_features:
#    X_train[col] = X_train[col].astype(str)
#    X_valid[col] = X_valid[col].astype(str)

#for col in binary_features:
#    X_train[col] = X_train[col].astype(bool)
#    X_valid[col] = X_valid[col].astype(bool)

In [20]:
# train model using lazy classifier and dont normalize data binary_features and categorical_features
# use F1 score as metric to rank models
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_valid, y_train, y_valid)

models.sort_values('F1 Score', ascending=False)

 93%|████████████████████████████████████████   | 27/29 [00:03<00:00,  7.51it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000341 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1987
[LightGBM] [Info] Number of data points in the train set: 2430, number of used features: 15
[LightGBM] [Info] Start training from score -1.006675
[LightGBM] [Info] Start training from score -0.723225
[LightGBM] [Info] Start training from score -2.839819
[LightGBM] [Info] Start training from score -4.962433
[LightGBM] [Info] Start training from score -4.184729
[LightGBM] [Info] Start training from score -3.668512
[LightGBM] [Info] Start training from score -3.141686
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

100%|███████████████████████████████████████████| 29/29 [00:04<00:00,  6.68it/s]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
ExtraTreesClassifier,0.79,0.56,None,0.78,0.29
RandomForestClassifier,0.77,0.54,None,0.77,0.58
LGBMClassifier,0.76,0.54,None,0.76,0.38
BaggingClassifier,0.73,0.50,None,0.73,0.19
LogisticRegression,0.73,0.42,None,0.71,0.07
LabelPropagation,0.70,0.51,None,0.70,0.33
LabelSpreading,0.70,0.51,None,0.70,0.43
KNeighborsClassifier,0.70,0.50,None,0.69,0.06
DecisionTreeClassifier,0.69,0.52,None,0.69,0.04


In [23]:
X_train

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Soil_Type,Wilderness_Area
701,2732.00,22.00,12.00,90.00,1.00,1360.00,212.00,214.00,137.00,624.00,Soil_Type_31,Wilderness_Area_3
1129,3161.00,233.00,7.00,420.00,59.00,162.00,208.00,247.00,176.00,2900.00,Soil_Type_23,Wilderness_Area_3
9632,3132.00,255.00,17.00,636.00,211.00,3144.00,179.00,248.00,207.00,2405.00,Soil_Type_31,Wilderness_Area_3
8176,2856.00,34.00,18.00,124.00,35.00,2382.00,215.00,199.00,114.00,2704.00,Soil_Type_29,Wilderness_Area_1
6595,3278.00,330.00,15.00,752.00,60.00,3565.00,183.00,220.00,176.00,1230.00,Soil_Type_22,Wilderness_Area_3
...,...,...,...,...,...,...,...,...,...,...,...,...
5368,2881.00,57.00,13.00,240.00,4.00,484.00,228.00,212.00,115.00,1771.00,Soil_Type_29,Wilderness_Area_1
3538,3188.00,56.00,7.00,247.00,13.00,4163.00,225.00,225.00,134.00,3070.00,Soil_Type_19,Wilderness_Area_1
3626,2812.00,86.00,14.00,150.00,0.00,3274.00,239.00,216.00,105.00,2564.00,Soil_Type_12,Wilderness_Area_1
4229,3092.00,3.00,19.00,459.00,1.00,1451.00,191.00,201.00,144.00,3571.00,Soil_Type_24,Wilderness_Area_3


In [27]:
# also try xgboost
from xgboost import XGBClassifier

# create preprocessing pipeline where numerical features are scaled and categorical features are one-hot encoded
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

numerical_transformer = Pipeline(steps=[('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, ['Soil_Type', 'Wilderness_Area'])
    ])

# create model pipeline where preprocessing is done and then the model is trained
model = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', XGBClassifier())])

# fit model
model.fit(X_train, y_train-1)

# predict on validation set
preds = model.predict(X_valid)

# calculate f1 score
from sklearn.metrics import f1_score

f1_score(y_valid-1, preds, average='weighted')


0.7610663222186763

In [40]:
# try a multi layer neural network with drop out, batch normalization and early stopping
# make the number number of units in the hidden layers a hyperparameter
# make the number of hidden layers a hyperparameter
# include an attention layer in the beginning
# use pytorch 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import f1_score

# create dataset class
class ForestDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.long)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
# create model class and make the number of hidden layers and number of units in each layer a hyperparameter
class ForestModel(nn.Module):
    def __init__(self, n_features, n_classes, n_hidden_layers, n_units, dropout=0.2):
        super(ForestModel, self).__init__()
        self.n_features = n_features
        self.n_classes = n_classes
        self.n_hidden_layers = n_hidden_layers
        self.n_units = n_units
        
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(self.n_features, self.n_units))
        self.layers.append(nn.BatchNorm1d(self.n_units))
        self.layers.append(nn.Dropout(dropout))
        self.layers.append(nn.ReLU())
        
        for i in range(self.n_hidden_layers):
            self.layers.append(nn.Linear(self.n_units, self.n_units))
            self.layers.append(nn.BatchNorm1d(self.n_units))
            self.layers.append(nn.Dropout(dropout))
            self.layers.append(nn.ReLU())
            
        self.layers.append(nn.Linear(self.n_units, self.n_classes))
        
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [50]:
# preprocess data
preprocessor.fit(X_train)

X_train_preprocessed = preprocessor.transform(X_train).toarray()
X_valid_preprocessed = preprocessor.transform(X_valid).toarray()

In [51]:
X_train_preprocessed

array([[-0.85210292, -1.18296971, -0.27244178, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.69963117,  0.73559923, -0.97643319, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.59473539,  0.93563959,  0.43154963, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-0.56273526, -0.60103411,  0.00915479, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.45005156, -1.35573184,  0.71314619, ...,  0.        ,
         1.        ,  0.        ],
       [-0.25166502,  0.82652667, -0.1316435 , ...,  0.        ,
         1.        ,  0.        ]])

In [53]:
# create train and validation dataloaders
train_dataset = ForestDataset(X_train_preprocessed, y_train)
valid_dataset = ForestDataset(X_valid_preprocessed, y_valid)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)

# create model
model = ForestModel(n_features=X_train_preprocessed.shape[1], n_classes=7, n_hidden_layers=3, n_units=32)

# create optimizer
optimizer = Adam(model.parameters(), lr=0.001)

# create learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

# create loss function
criterion = nn.CrossEntropyLoss()

# train model with 5000 epochs and early stopping
epochs = 5000
patience = 20
best_valid_loss = np.inf
counter = 0

for epoch in range(epochs):
    train_loss = 0.0
    valid_loss = 0.0
    
    # train model
    model.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target-1)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
        
    # validate model
    model.eval()
    for data, target in valid_loader:
        output = model(data)
        loss = criterion(output, target-1)
        valid_loss += loss.item() * data.size(0)
        
    # calculate average losses
    train_loss = train_loss / len(train_loader.sampler)
    valid_loss = valid_loss / len(valid_loader.sampler)
    
    # update learning rate scheduler
    scheduler.step(valid_loss)
    
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        valid_loss
        ))
    
    # early stopping
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            break

# predict on validation set
model.eval()
preds = []
for data, target in valid_loader:
    output = model(data)
    preds.append(output.detach().numpy())
preds = np.concatenate(preds)
preds = np.argmax(preds, axis=1)

# calculate f1 score
f1_score(y_valid-1, preds, average='weighted')

Epoch: 1 	Training Loss: 1.778488 	Validation Loss: 1.544720
Epoch: 2 	Training Loss: 1.336453 	Validation Loss: 1.147701
Epoch: 3 	Training Loss: 1.116099 	Validation Loss: 0.945994
Epoch: 4 	Training Loss: 1.006441 	Validation Loss: 0.846125
Epoch: 5 	Training Loss: 0.921695 	Validation Loss: 0.787852
Epoch: 6 	Training Loss: 0.871553 	Validation Loss: 0.750889
Epoch: 7 	Training Loss: 0.851257 	Validation Loss: 0.717989
Epoch: 8 	Training Loss: 0.816663 	Validation Loss: 0.708737
Epoch: 9 	Training Loss: 0.793385 	Validation Loss: 0.692725
Epoch: 10 	Training Loss: 0.797722 	Validation Loss: 0.685052
Epoch: 11 	Training Loss: 0.767407 	Validation Loss: 0.675190
Epoch: 12 	Training Loss: 0.764796 	Validation Loss: 0.661481
Epoch: 13 	Training Loss: 0.746931 	Validation Loss: 0.661412
Epoch: 14 	Training Loss: 0.765914 	Validation Loss: 0.657719
Epoch: 15 	Training Loss: 0.754191 	Validation Loss: 0.644901
Epoch: 16 	Training Loss: 0.747649 	Validation Loss: 0.644737
Epoch: 17 	Traini

0.7409079731859195

We decided to tune the parameters of the **ExtraTreesClassifier**, **RandomForestClassifier**, **LGBMClassifier**, **BaggingClassifier**, **XGBoost** and the **Neural network**

In [54]:
# find best hyperparameters for each of the chosen models using gp_minimize
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from sklearn.model_selection import cross_val_score

# create preprocessing pipeline where numerical features are scaled and categorical features are one-hot encoded
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

numerical_transformer = Pipeline(steps=[('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, ['Soil_Type', 'Wilderness_Area'])
    ])

# define pipeline pipeline lambda function for any classifier
pipe = lambda classifier, params: Pipeline(steps=[('preprocessor', preprocessor),
                                          ('classifier', classifier(**params))])

In [55]:
# define search space for each classifier

# random forest
rf_space = [
    Integer(10, 1000, name='n_estimators'),
    Integer(2, 10, name='max_depth'),
    Integer(2, 10, name='min_samples_split'),
    Integer(1, 10, name='min_samples_leaf'),
    Categorical(['gini', 'entropy'], name='criterion')
]

# xgboost
xgb_space = [
    Integer(10, 1000, name='n_estimators'),
    Real(0.001, 1.0, name='learning_rate'),
    Integer(2, 10, name='max_depth'),
    Real(0.0, 1.0, name='subsample'),
    Real(0.0, 1.0, name='colsample_bytree'),
    Real(0.0, 1.0, name='colsample_bylevel')
]

# lightgbm
lgbm_space = [
    Integer(10, 1000, name='n_estimators'),
    Real(0.001, 1.0, name='learning_rate'),
    Integer(2, 10, name='max_depth'),
    Real(0.0, 1.0, name='subsample'),
    Real(0.0, 1.0, name='colsample_bytree'),
    Real(0.0, 1.0, name='colsample_bylevel')
]

# bagging classifier
bagging_space = [
    Integer(10, 1000, name='n_estimators'),
    Real(0.0, 1.0, name='max_samples'),
    Real(0.0, 1.0, name='max_features')
]

# extra trees classifier
extra_trees_space = [
    Integer(10, 1000, name='n_estimators'),
    Integer(2, 10, name='max_depth'),
    Integer(2, 10, name='min_samples_split'),
    Integer(1, 10, name='min_samples_leaf'),
    Categorical(['gini', 'entropy'], name='criterion')
]

# neural network
nn_space = [
    Integer(1, 10, name='n_hidden_layers'),
    Integer(1, 100, name='n_units'),
    Real(0.0, 1.0, name='dropout')
]

In [56]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier


# hyperparameter search for each classifier

# random forest
@use_named_args(rf_space)
def rf_objective(**params):
    clf = pipe(RandomForestClassifier, params)
    return -np.mean(cross_val_score(clf, X_train, y_train, cv=5, n_jobs=-1, scoring='f1_weighted'))

# xgboost
@use_named_args(xgb_space)
def xgb_objective(**params):
    clf = pipe(XGBClassifier, params)
    return -np.mean(cross_val_score(clf, X_train, y_train, cv=5, n_jobs=-1, scoring='f1_weighted'))

# lightgbm
@use_named_args(lgbm_space)
def lgbm_objective(**params):
    clf = pipe(LGBMClassifier, params)
    return -np.mean(cross_val_score(clf, X_train, y_train, cv=5, n_jobs=-1, scoring='f1_weighted'))

# bagging classifier
@use_named_args(bagging_space)
def bagging_objective(**params):
    clf = pipe(BaggingClassifier, params)
    return -np.mean(cross_val_score(clf, X_train, y_train, cv=5, n_jobs=-1, scoring='f1_weighted'))

# extra trees classifier
@use_named_args(extra_trees_space)
def extra_trees_objective(**params):
    clf = pipe(ExtraTreesClassifier, params)
    return -np.mean(cross_val_score(clf, X_train, y_train, cv=5, n_jobs=-1, scoring='f1_weighted'))


In [57]:
rf_gp_result = gp_minimize(rf_objective, rf_space, n_calls=50, random_state=42, verbose=True)

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 3.3286
Function value obtained: -0.5493
Current minimum: -0.5493
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.9929
Function value obtained: -0.6614
Current minimum: -0.6614
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.7776
Function value obtained: -0.6930
Current minimum: -0.6930
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 1.9878
Function value obtained: -0.6900
Current minimum: -0.6930
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.9001
Function value obtained: -0.3650
Current minimum: -0.6930
Iteration No: 6 started. 